<a href="https://colab.research.google.com/github/Sohaila24/GenAI-Basic-Learnings/blob/main/LlamaIndex_API_using_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.2/790.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [8]:
import os
import openai

from dotenv import load_dotenv
load_dotenv() # It must be before llama_index import

from llama_index.core import GPTVectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI


from llama_index.core import ServiceContext, StorageContext, load_index_from_storage

In [9]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [10]:
def build_storage(data_dir, persist_dir):
    documents = SimpleDirectoryReader(data_dir).load_data()
    index = GPTVectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir)
    return index

In [11]:
def read_from_storage(persist_dir):
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    return load_index_from_storage(storage_context)

In [12]:
def get_storage_index():
    persist_dir = os.environ.get("STORAGE_PATH")
    data_dir =  os.environ.get("DATA_PATH")
    index = None

    if os.path.exists(persist_dir):
        index = read_from_storage(persist_dir)
    else:
        index = build_storage(data_dir, persist_dir)

    return index

In [13]:
def get_query_engine():

    llm = OpenAI(model="gpt-3.5-turbo-16k", temperature=0.2, max_tokens=8000, streaming=True)
    embed_model = OpenAIEmbedding()
    service_context = ServiceContext.from_defaults(llm=llm,embed_model=embed_model, chunk_size=512)
    #set_global_service_context(service_context)

    index = get_storage_index()
    query_engine = index.as_query_engine(service_context=service_context)
    return query_engine

In [14]:
def query_llm(q):

    query_engine = get_query_engine()

    response = query_engine.query(q)

    print(response)
    return(response)

In [17]:
!pip install Flask

In [18]:
from flask import Flask, request, json, abort

PREFIX = "/v1" # define prefix here or import

app = Flask(__name__) # Create a Flask app instance

In [19]:
@app.route(f'{PREFIX}/query',  methods=['POST'])
def query():
    content = request.get_json(silent=True)
    q= content['query']

    try:
         res = gptindex.query_llm(q)

         return json.dumps({"response": str(res)})
    except Exception as err:
        print(err)
        abort(404)